# Equity Hedging Analysis Notebook

This notebook runs analysis on the Strategy and Allocation Equity Hedge Portfolio

## Import libraries

In [39]:
import pandas as pd
import ipywidgets as widgets
import datapane as dp 
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual

new_strat = False
notional_weights = []
weighted = [True, False]

## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [22]:
equity_bmk = 'SPTR'
include_fi = False
strat_drop_list = ['Vortex','99%/90% Put Spread']
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## View Returns data

Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [23]:
freq_list = ['Daily', 'Weekly', 'Monthly','Quarterly', 'Yearly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(returns[frequency])

interactive(children=(Dropdown(description='frequency', options=('Daily', 'Weekly', 'Monthly', 'Quarterly', 'Y…

## Add new strategy

**Run this code if you want to add a new strategy**
* Import the daily index/returns
* add it to the Equity hedge returns data

In [ ]:
git 

## Get and display Notional Weights

In [24]:
#get notional weights
notional_weights = dm.get_notional_weights(returns['Monthly'])
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

notional value (Billions) for SPTR: 19
notional value (Billions) for Down Var: 1
notional value (Billions) for VOLA: 1
notional value (Billions) for Dynamic Put Spread: 1
notional value (Billions) for VRR: 1
notional value (Billions) for GW Dispersion: 1
notional value (Billions) for Corr Hedge: .25
notional value (Billions) for Def Var: 1


,SPTR,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


# Compute Correlations

In [5]:
corr_freq_list = ['Daily', 'Weekly', 'Monthly']
corr_dict = summary.get_corr_data(returns, corr_freq_list, weighted, notional_weights, include_fi)

## Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **weighted (True or False)** - Show Weighted Hedges or not
* **corr** - Show:
    * **corr** - full history correaltions
    * **corr_down** - risk seeking benchmark downside returns correlations or 
    * **corr_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [6]:
@interact
def display_correlations(frequency=corr_freq_list, weighted=weighted,
                         corr=['corr', 'corr_down', 'corr_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict[frequency]
    corr_df = data[weighted][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

interactive(children=(Dropdown(description='frequency', options=('Daily', 'Weekly', 'Monthly'), value='Daily')…

## Compute Return Stats and Hedge Metrics

In [35]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(returns,analytics_freq_list,weighted,notional_weights,include_fi,new_strat)

notional value (Billions) for SPTR: 19
notional value (Billions) for Down Var: 1
notional value (Billions) for VOLA: 1
notional value (Billions) for Dynamic Put Spread: 1
notional value (Billions) for VRR: 1
notional value (Billions) for GW Dispersion: 1
notional value (Billions) for Corr Hedge: .25
notional value (Billions) for Def Var: 1


## Display Return Stats and Hedge Metrics

Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **weighted (True or False)** - Show Weighted Strats and Hedges or not
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [36]:
@interact
def display_analytics(frequency=analytics_freq_list, weighted=weighted, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[weighted]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(returns,notional_weights=notional_weights, weighted=weighted)

## Display Historical Sell Offs

Display Hisorical Sell Offs data by:
* **weighted (True or False)** - Show Weighted Hedges or not

In [ ]:
@interact
def display_selloffs(weighted=weighted):
    df_hist = hist_dict[weighted]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile Analysis

In [ ]:
quintile_df = summary.get_quintile_data(returns, notional_weights,weighted=True)
quintile_df.style.format("{:.2%}")

## Display Quintile Analysis

Display Quintile Analysis Bar Chart:
* Compare hedging strategies (Weighted Hedges as well) to the Equity Benchmark's returns

In [ ]:
@interact
def display_quintile_analysis(strat=list(quintile_df.select_dtypes('float').columns)[1:]):
    df = quintile_df.copy()
    for col in df.columns:
        if not (col == equity_bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    fig = px.bar(df,barmode='group',title =  "Quintile Analysis")
    fig.show()

## Compute Annual Dollar Returns

In [ ]:
#clean index to display year only
annual_dollar_returns = summary.get_annual_dollar_returns(returns, notional_weights)
fmt.get_dollar_ret_styler(annual_dollar_returns)

## Display Annual Dollar Returns

In [ ]:
@interact
def display_annual_dollar_returns(strat_1=list(annual_dollar_returns.select_dtypes('float').columns)
      ,strat_2=list(annual_dollar_returns.select_dtypes('float').columns)[1:]):
    df = annual_dollar_returns.copy()
    for col in df.columns:
        if not (col == strat_1 or col ==strat_2):
            df.drop([col], axis=1, inplace=True)
    fig = px.bar(df,barmode='group',title = "Annual $ Returns")
    fig.show()

## Create Report

Run this code below to export analysis into excel spreadsheet
* **equity_hedge_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [9]:
equity_hedge_report = 'equity_hedge_analysis_q12021'
selloffs = True
rp.get_equity_hedge_report(equity_hedge_report, returns, notional_weights, include_fi, new_strat, weighted[0], selloffs)

Computing Monthly Analytics...
Computing Weekly Analytics...
Computing Historical SellOffs...
